In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sqlite3
import glob
import subprocess
import math

In [2]:
dem = pd.read_csv('demand_winter_work.tsv', header = None, sep = ' ')

C1 = dem.iloc[0:34542]
C2 = dem.iloc[34542:69084].reset_index()
C3 = dem.iloc[69084:103626].reset_index()
C4 = dem.iloc[103626:138168].reset_index()
C5 = dem.iloc[138168:172710].reset_index()
C6 = dem.iloc[172710:207252].reset_index()
C7 = dem.iloc[207252:241794].reset_index()
C8 = dem.iloc[241794:276336].reset_index()

C = pd.DataFrame()
C['C1'] = C1[2]
C['C2'] = C2[2]
C['C3'] = C3[2]
C['C4'] = C4[2]
C['C5'] = C5[2]
C['C6'] = C6[2]
C['C7'] = C7[2]
C['C8'] = C8[2]
#C['sum'] = C.sum(axis=1)
#C['step'] = C1[0]
C

,C1,C2,C3,C4,C5,C6,C7,C8
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...
34537,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
34538,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
34539,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
34540,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [4]:
T = pd.DataFrame()
T['max'] = C.sum(axis=1)
T['max'].idxmax()

16290

In [5]:
C.iloc[16290:16291]

,C1,C2,C3,C4,C5,C6,C7,C8
16290,103.67,115.48,58.75,140.34,0.0,0.0,0.0,0.0


In [10]:
C.iloc[16290]['C5'] = 195.44
C.iloc[16290]['C6'] = 195.44
C.iloc[16290]['C7'] = 195.44
C.iloc[16290]['C8'] = 195.44

T = pd.DataFrame()
T['max'] = C.sum(axis=1)
T['max'].max()

1200.0

In [11]:
time = np.arange(0,34542,1)
ind = []

for i in range(len(time)):
    ind.append('m' + str(time[i]))

E = pd.concat([C['C1'], C['C2'],C['C3'],C['C4'],C['C5'], C['C6'],C['C7'],C['C8']], ignore_index = True)
indall = ind * 8
len(indall)
len(E)
charger = ['C1']*34542 + ['C2']*34542 + ['C3']*34542 + ['C4']*34542 + ['C5']*34542 + ['C6']*34542 + ['C7']*34542 + ['C8']*34542

a = indall
b = charger
c = []

for i in range(len(b)):
    c.append(str(a[i] + '.' + str(b[i])))
    
df4 = pd.DataFrame()
df4['timestep'] = c
df4['load'] = E

df4

,timestep,load
0,m0.C1,0.0
1,m1.C1,0.0
2,m2.C1,0.0
3,m3.C1,0.0
4,m4.C1,0.0
...,...,...
276331,m34537.C8,0.0
276332,m34538.C8,0.0
276333,m34539.C8,0.0
276334,m34540.C8,0.0


In [12]:
#df4.to_csv('demand_winter1200', sep=',',index = False)

In [2]:

with open("EVCS_15_G.gms","r") as f:
    list_of_lines = f.readlines()
    
list_of_lines[35]

'$include demand_winter_350distributed.tsv\n'

In [2]:

with open("EVCS_15_G.gms","r") as f:
    list_of_lines = f.readlines()
    
list_of_lines[35] = "$include demand_winter1200.tsv\n"

print(list_of_lines[35])
    
with open("EVCS_15_G.gms", "w") as f:
    f.writelines(list_of_lines)
        
subprocess.run(["gams", "EVCS_15_G.gms", "logOption", "2"])
    
subprocess.run(["move", "EVCS_15_G_results.db", "EVCS_15_G1_1200_results.db"], shell = True)


$include demand_winter1200.tsv



CompletedProcess(args=['move', 'EVCS_15_G_results.db', 'EVCS_15_G1_1200_results.db'], returncode=0)

In [7]:
#Extract result data from all result databases and create dataframes for each variable

grid_p = pd.DataFrame()
grid_ex_p = pd.DataFrame()
grid_tot_p = pd.DataFrame()

soc_pen = pd.DataFrame()

grid_e = pd.DataFrame()
grid_ex_e = pd.DataFrame()

ESS_lvl = pd.DataFrame()
ESS_p = pd.DataFrame()

sclrs = pd.DataFrame()

for filename in glob.glob('EVCS_15_G1_*.db'):
    con = sqlite3.connect(filename)
    
    grid_power = pd.read_sql_query("SELECT * FROM var_grid_power", con)
    grid_extra_power = pd.read_sql_query("SELECT * FROM var_grid_ex_power", con)
    grid_total_power = pd.read_sql_query("SELECT * FROM var_grid_total_power", con)
    
    grid_energy = pd.read_sql_query("SELECT * FROM var_grid_energy", con)
    grid_extra_energy = pd.read_sql_query("SELECT * FROM var_grid_ex_energy", con)
    
    ESS_level = pd.read_sql_query('SELECT * FROM var_ESS_level', con)
    ESS_power = pd.read_sql_query('SELECT * FROM var_ESS_power', con)
    
    scalars =  pd.read_sql_query('SELECT * FROM scalarvariables', con)
    
    soc_penalty = pd.read_sql_query('SELECT * FROM var_SOC_pen', con)
    
    con.close()
    
    grid_p[filename[11:-11]] = grid_power['level'].copy()
    grid_ex_p[filename[11:-11]] = grid_extra_power['level'].copy()
    grid_tot_p[filename[11:-11]] = grid_total_power['level'].copy()
    
    grid_e[filename[11:-11]] = grid_energy['level'].copy()
    grid_ex_e[filename[11:-11]] = grid_extra_energy['level'].copy() 
    
    ESS_lvl[filename[11:-11]] = ESS_level['level'].copy()
    ESS_p[filename[11:-11]] = ESS_power['level'].copy()  
    
    sclrs[filename[11:-11]] = scalars['level'].copy()
    
    soc_pen[filename[11:-11]] = soc_penalty['level'].copy()


In [8]:
sclrs = sclrs.set_index(scalars['name'])
sclrs.iloc[-1] = sclrs.iloc[-1] * 1.25
#cols = sclrs.columns.tolist()
#cols = [cols[1] , cols[-1] , cols[0]]
#sclrs = sclrs[cols]
sclrs

,1200,300,350,low,OG
name,,,,,
var_ess_rated_power,173.962500,173.962500,173.962500,173.962500,173.968750
var_grid_maxpower1,222.597500,150.824462,176.037500,222.597500,222.591250
var_grid_maxpower2,1026.037500,241.911652,264.028476,223.340906,264.807500
var_grid_maxpower3,180.003200,180.003200,180.003200,180.003200,180.000000
var_grid_maxpower4,225.507500,152.178750,176.037500,225.507500,225.501250
var_ESS_cost,30617.400000,30617.400000,30617.400000,30617.400000,30618.500000
var_electricity_cost,141582.307451,140725.695900,140867.481838,138121.923830,141094.161756
var_grid_tariff_winter,123851.454217,58784.321299,63770.206695,67568.492917,70553.808901
var_energy_tax,3603.447889,3602.265477,3602.265477,3585.892090,3602.265477
